In [1]:
import pandas as pd

# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

## Descargar datos de fuente oficial (Min Econ)

In [2]:
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)

url_fuente = 'https://www.economia.gob.ar/download/infoeco/apendice3a.xlsx'
response = requests.get(url_fuente, verify = False)
open('./datos/apendice3a.xlsx', "wb").write(response.content)

2254414

In [3]:
## Leer Excel
EPH_puntual = pd.read_excel('./datos/apendice3a.xlsx', sheet_name = 'EPH Puntual')
TD = pd.read_excel('./datos/apendice3a.xlsx', sheet_name = 'TD 03-')

## Elegir Hoja y indicar nombre de variables a extraer

In [4]:
datos = TD
serie = '45.2_'

# O bien, por ejemplo, para tomar la de EPH puntual:
# datos = EPH_puntual
# serie = '42.3_'

## Extraer dataframe de la hoja de Excel

In [5]:
mask = datos.apply(lambda x: x.astype(str).str.contains(serie))

df = datos[mask.cumsum() == True].dropna(axis = 1, how = 'all').dropna()
df, df.columns = df[1:] , df.iloc[0]
col_tiempo = df.columns[0]

df[col_tiempo] = df[col_tiempo].str.split('.').apply(lambda x: x[1]+x[0])
df[col_tiempo] = df[col_tiempo].str.replace(' ', '')
df[col_tiempo] = df[col_tiempo].str.replace('III', 'Q3').str.replace('II', 'Q2').str.replace('IV', 'Q4').str.replace('I', 'Q1')

df[col_tiempo] = pd.to_datetime(df[col_tiempo])

df = df.set_index(col_tiempo)
df.tail()

30,45.2_ECTDT_0_T_33,45.2_ECTDTG_0_T_37,45.2_ECTDTCU_0_T_38,45.2_ECTDTNE_0_T_42,45.2_ECTDTNO_0_T_42,45.2_ECTDTRP_0_T_49,45.2_ECTDTP_0_T_43,45.2_ECTDTC_0_T_38,45.2_ECTDTPG_0_T_46,45.2_ECTDTGM_0_T_46,...,45.2_ECTDTGSF_0_T_47,45.2_ECTDTRC_0_T_44,45.2_ECTDTSR_0_T_44,45.2_ECTDTSN_0_T_45,45.2_ECTDTCR_0_T_52,45.2_ECTDTN_0_T_41,45.2_ECTDTRG_0_T_46,45.2_ECTDTURG_0_T_52,45.2_ECTDTRT_0_T_47,45.2_ECTDTVCP_0_T_57
45.2_IT_0_T_13,,,,,,,,,,,,,,,,,,,,,
2021-01-01,0.102,0.11,0.066,0.061,0.099,0.103,0.084,0.079,0.119,0.082,...,0.09,0.068,0.095,0.086,0.044,0.109,0.09,0.123,0.061,0.039
2021-04-01,0.096009,0.101938,0.06825,0.060731,0.087344,0.104124,0.073916,0.090066,0.105247,0.085309,...,0.089013,0.072951,0.133491,0.081496,0.026347,0.081887,0.082397,0.099239,0.10727,0.039952
2021-07-01,0.082,0.089,0.07,0.045,0.062,0.088,0.055,0.07,0.095,0.084,...,0.086,0.059,0.099,0.065,0.033,0.07,0.057,0.048,0.085,0.015
2021-10-01,0.07,0.074,0.06,0.046,0.058,0.074,0.045,0.046,0.082,0.068,...,0.063,0.057,0.069,0.074,0.024,0.059,0.037,0.054,0.056,0.016
2022-01-01,0.07,0.073,0.054,0.051,0.066,0.074,0.048,0.049,0.08,0.065,...,0.071,0.036,0.071,0.068,0.021,0.074,0.061,0.046,0.038,0.01


### Tomar raiz compartida por las variables extraidas

In [6]:
from difflib import SequenceMatcher

## Nombre de variables compartido por todas las series del dataset actual (lo llamamos raiz):
raiz_ = df.columns[0]
for col in df.columns:
    match = SequenceMatcher(None, raiz_, col).find_longest_match(0, len(raiz_), 0, len(col))
    next_ = raiz_[match.a:match.a + match.size]
    raiz_ = next_
    
print(raiz_)

45.2_ECTDT


## Guardar CSV

In [7]:
df.to_csv('./datos/'+raiz_+'.csv')